In [1]:
import pandas as pd
import numpy as np
import seaborn as sbs
import matplotlib.pyplot as plt

In [2]:
train_data =  pd.read_csv('train.csv')
test_data = pd.read_csv("test.csv")

# Feature Engineering

Remove Null Values

In [4]:
train_data.isnull().sum()[train_data.isnull().sum() > 0]
test_data.isnull().sum()[test_data.isnull().sum() > 0]

,0


In [ ]:
num_cont = ["LotFrontage", "MasVnrArea", "GarageYrBlt"]

for col in num_cont:
    mean_value = train_data[col].mean()   # mean from train only
    train_data[col].fillna(mean_value, inplace=True)
    test_data[col].fillna(mean_value, inplace=True)  # use same mean



/tmp/ipython-input-4126292244.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data[col].fillna(mean_value, inplace=True)
/tmp/ipython-input-4126292244.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [ ]:
train_data.isnull().sum()[train_data.isnull().sum() > 0]

,0
Alley,1369
MasVnrType,872
BsmtQual,37
BsmtCond,37
BsmtExposure,38
BsmtFinType1,37
BsmtFinType2,38
Electrical,1
FireplaceQu,690
GarageType,81


Label Encoding

In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in train_data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col].astype(str))

    # Handle unseen labels in test set
    test_data[col] = test_data[col].astype(str)
    test_data[col] = test_data[col].apply(lambda x: x if x in le.classes_ else 'Unknown')

    # Add "Unknown" as a new possible class
    le.classes_ = np.append(le.classes_, 'Unknown')
    test_data[col] = le.transform(test_data[col])

    label_encoders[col] = le


In [ ]:
train_data.to_excel("standard.xlsx", index=False)

Standard Encoding

In [6]:
to_standardize = [
    "annual_income",
    "credit_score",
    "loan_amount",
    "interest_rate"
]

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_data[to_standardize] = scaler.fit_transform(train_data[to_standardize])
test_data[to_standardize] = scaler.transform(test_data[to_standardize])

In [9]:
train_data.to_csv("standardation.csv", index=False)

suuu

Splitting the Data

In [10]:
X_train = train_data.iloc[:,1:-1]
y_train = train_data.iloc[:,-1]
X_test = test_data.iloc[:,1:]

In [ ]:
#y_train = y_train.reshape(-1, 1)

Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# parameters ={
#     "criterion"    : ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
#     "max_depth"    : [2,6,8,10],
#     "ccp_alpha"    : [0,1,4,7,10,50,100]

# }

parameters = { "n_estimators": [5,10,20,30],
    "max_depth": [5,6,7,8,9],
    "criterion": ['squared_error', 'poisson'],
    "ccp_alpha"    : [0,1,4,7,10]
 }

In [ ]:
treemodel = RandomForestRegressor()
cv = GridSearchCV(treemodel,param_grid = parameters,cv=4,scoring='r2')

In [ ]:
cv.fit(X_train,y_train)

GridSearchCV(cv=4, estimator=RandomForestRegressor(),
             param_grid={'ccp_alpha': [0, 1, 4, 7, 10],
                         'criterion': ['squared_error', 'poisson'],
                         'max_depth': [2, 3, 4, 5],
                         'n_estimators': [5, 10, 20, 30]},
             scoring='r2')

# XG Boosting

In [36]:

from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()
regressor.fit(X_train,y_train)


RandomForestRegressor()

In [39]:
y_pred = regressor.predict(X_test)
y_pred

array([0.92, 0.99, 0.39, ..., 0.99, 0.99, 0.94])

In [40]:
import pandas as pd
import numpy as np

y_pred = np.round(y_pred)

# Replace -0.0 with 0.0
y_pred[y_pred == -0.0] = 0.0


submission = pd.DataFrame({
    "id": range(593994, 848563),  # since end is exclusive
    "loan_paid_back": y_pred
})

# Save to CSV
submission.to_csv("RandomForest_Regressor_Loan.csv", index=False)
print("✅ File saved as xgboost_loan.csv")


✅ File saved as xgboost_loan.csv
